# Fixing Random Seed in Numba for Reproducible Results

This notebook demonstrates how to achieve reproducible random number generation in Numba-compiled functions. In standard Python, setting `np.random.seed()` once at the start ensures reproducibility across all subsequent random calls. However, in Numba's JIT-compiled code, random state management is isolated and requires special handling.

## The Problem
When using `@numba.jit`, setting `np.random.seed()` globally (before calling JIT functions) may not propagate into the compiled code's random state, leading to non-reproducible results.

## The Solution
Use `@numba.extending.register_jitable` to create a seed-setting function that can be called **once before** your JIT-compiled code runs, ensuring the JIT random state is initialized properly.

<a href="https://colab.research.google.com/github/Ziaeemehr/workshop_hpcpy/blob/main/notebooks/numba/numba_fix_seed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
from numba import jit
from numba.extending import register_jitable

# Global seed for non-JIT code
np.random.seed(42)

## Registering a Jitable Seed Function

The `@register_jitable` decorator allows a function to be called from JIT-compiled code while still executing in JIT mode.

In [7]:
@register_jitable
def set_seed_compact(x):
    """Set the random seed in a way that works in JIT-compiled functions."""
    np.random.seed(x)

## JIT Function that Uses Random Numbers

This function generates random numbers. The seed should be set **before** calling this function, not inside it.

In [8]:
@jit(nopython=True)
def get_random():
    """Generate random numbers using the JIT random state."""
    return np.random.rand(3)

## Proper Reproducibility: Set Seed Once

The key to reproducibility is to set the seed **once** before running your code. This initializes the random state for subsequent calls.

In [9]:
# Initialize the seed once in a JIT context
@jit(nopython=True)
def initialize_seed():
    set_seed_compact(42)

# Set the seed once
initialize_seed()

# Now each call will produce different numbers, but the sequence is reproducible
print("First call:", get_random())
print("Second call:", get_random())
print("Third call:", get_random())

First call: [0.37454012 0.95071431 0.73199394]
Second call: [0.59865848 0.15601864 0.15599452]
Third call: [0.05808361 0.86617615 0.60111501]


## Demonstrating Reproducibility

If we re-initialize the seed with the same value and run the same sequence, we get identical results.

In [10]:
# Re-initialize with the same seed
initialize_seed()

# The sequence repeats exactly
print("First call (repeat):", get_random())
print("Second call (repeat):", get_random())
print("Third call (repeat):", get_random())

First call (repeat): [0.37454012 0.95071431 0.73199394]
Second call (repeat): [0.59865848 0.15601864 0.15599452]
Third call (repeat): [0.05808361 0.86617615 0.60111501]


## Comparison with Regular Python

Regular Python uses `np.random.seed()` which sets the global random state. The same principle applies: set the seed once, get a reproducible sequence.

In [11]:
# Regular Python with proper seed initialization
np.random.seed(42)

def get_random2():
    """Regular Python function using global random state."""
    return np.random.rand(3)

print("Regular Python - First:", get_random2())
print("Regular Python - Second:", get_random2())

# Re-seed to demonstrate reproducibility
np.random.seed(42)
print("Regular Python - First (repeat):", get_random2())
print("Regular Python - Second (repeat):", get_random2())

Regular Python - First: [0.37454012 0.95071431 0.73199394]
Regular Python - Second: [0.59865848 0.15601864 0.15599452]
Regular Python - First (repeat): [0.37454012 0.95071431 0.73199394]
Regular Python - Second (repeat): [0.59865848 0.15601864 0.15599452]


## Why This Matters

- **Reproducibility**: Essential for scientific computing, testing, and debugging.
- **JIT Compatibility**: Standard `np.random.seed()` doesn't work reliably in compiled Numba code.
- **Performance**: Using jitable functions maintains JIT performance benefits.

For more details, see the [Numba documentation on random numbers](https://numba.readthedocs.io/en/stable/reference/numpysupported.html#random).